In [0]:
# 문제 정의 : 각 책 별로 올해 상반기(1월~6월) 동안 베스트셀러 순위 변동 순위를 알고 싶었습니다.
# 다음 책에서 주차별 베스트셀러 종합 10위를 뽑아서 pandas의 dataframe에 담았습니다.
from urllib import request
import pandas as pd
import html5lib
# url을 변경하여 각 0월 0주차 주소에 접근하고자 했습니다.
# 처음 들어가는 주소는 http://book.daum.net/bestseller/list.do입니다.
url_start = "http://book.daum.net/bestseller/list.do?categoryID=SPDT00000DM&ymd=20180"
# 5개의 column으로 title은 책 제목, author은 저자, rank는 순위, month와 week는 0월 0주차를 표현했습니다.
df = pd.DataFrame(columns=['title','author','rank','month','week'])

In [4]:
# url에 들어갈 *월 @주차를 "*@"이라는 문자열로 반환하는 함수입니다. 
# 예를 들어 다음 책에서는 ymd=2018064로 6월 4주차를 표현하므로 뒷 숫자 2자리를 바꿔서 url를 새롭게 구성하고, 그 url로 크롤링을 진행합니다.
def get_dates():
    dates = []
    # 1월 1주차부터 6월 5주차까지 뽑습니다.
    for month in range(1,7):
        for week in range(1,6):
            dates.append(str(month)+str(week))
    return dates
    
# html을 불러오는 부분을 함수로 정리했습니다.
def get_html(url):
    f = request.urlopen(url)
    html = f.read().decode('utf-8')
    return html

# crawling할 컨텐츠가 있는 부분들을 getchildren을 활용해 1차적으로 추출하여 body로 반환하는 함수입니다.
def get_body(dom):
    body_children = dom.getchildren()[1].getchildren()[2].getchildren()[2].getchildren()
    body = None
    for i in body_children:
        # 딕셔너리형로 반환하는 i.attrib에서 id 태그가 아예 없으면 오류가 나므로 if문으로 id라는 key가 있는지 확인
        if 'id' in i.attrib:
            # id라는 key의 값이 page_body인 태그를 찾고 body에 넣고, body를 반환합니다.
            if i.attrib['id'] == 'page_body' :
                body = i
    return body

# 베스트셀러 top 3개를 뽑는 함수입니다.
def get_book_top3(li):
    dic = { }
    title = li.getchildren()[2]
    dic['title'] = title.text
    # author이 아무것도 없는 경우가 있으면 IndexError가 나므로 ""으로 표시하는 예외 처리를 해줍니다.
    try :
        author = li.getchildren()[3].getchildren()[0]
        dic['author'] = author.text
    except IndexError:
        dic['author'] = ""
    return dic

# top 3개를 제외한 7개를 뽑는 함수입니다.
def get_book_rest7(li):
    dic = { }
    title = li.getchildren()[2].getchildren()[0].getchildren()[0].getchildren()[0]
    dic['title'] = title.text
    # 새로운 태그를 하나 중간에 더 끼워넣는 경우가 있어서 예외처리로 getchildren의 index 값을 변경해주었습니다.
    try: 
        author = li.getchildren()[2].getchildren()[1].getchildren()[0].getchildren()[0]
        dic['author'] = author.text
    except IndexError: 
        # top 3와 마찬가지로 저자가 없는 경우 Index 에러가 나므로 예외처리를 해줍니다.
        try: 
            author = li.getchildren()[2].getchildren()[1].getchildren()[1].getchildren()[0]
            dic['author'] = author.text
        except IndexError:
            dic['author'] = ""
    return dic
    
  
## main
for date in get_dates():
    # get_dates로 만든 월과 주차의 date 2자리로 url을 제작합니다.
    url = url_start+date
    html = get_html(url)
    dom = html5lib.parse(html)

    body = get_body(dom)
    # 2018년 5월 5주차처럼 존재하지 않는 주차가 있으면 IndexError가 나므로 예외처리를 해줘서 크롤링을 진행하지 않고 다음 주차로 바로 넘깁니다.
    try:
        ol = body.getchildren()[1].getchildren()[3].getchildren()[0].getchildren()
        ul = body.getchildren()[1].getchildren()[3].getchildren()[3].getchildren()[0]
    except IndexError:
        continue

    # top 3개를 뽑습니다.
    for i,li in enumerate(ol):
        dic = get_book_top3(li)
        dic['rank'] = i+1
        dic['month'] = date[0]
        dic['week'] = date[1]
        df = df.append(dic,ignore_index=True)

    # top 7개를 뽑습니다.
    for i,li in enumerate(ul):
        dic = get_book_rest7(li)
        dic['rank'] = i+4
        dic['month'] = date[0]
        dic['week'] = date[1]
        df = df.append(dic,ignore_index=True)
print(df)

                                        title        author rank month week
0               랩 걸: 나무, 과학 그리고 사랑-나무, 과학 그..         호프 자런    1     1    1
1               82년생 김지영(오늘의 젊은 작가 13)-조남주 ..           조남주    2     1    1
2               신경 끄기의 기술 (블랙커버 특별판) *배송주문 ..         마크 맨슨    3     1    1
3                         말 그릇-비울수록 사람을 더 채우는           김윤나    4     1    1
4                                  그대 눈동자에 건배      히가시노 게이고    5     1    1
5                          인생에서 너무 늦은 때란 없습니다  애나 메리 로버트슨..    6     1    1
6                                      언어의 온도           이기주    7     1    1
7    우리는 언젠가 만난다-나, 타인, 세계를 이어주는 40가지 눈부신 이야기           채사장    8     1    1
8                      역시 내 청춘 러브코메디는 잘못됐다 12       와타리 와타루    9     1    1
9                                 나미야 잡화점의 기적      히가시노 게이고   10     1    1
10              신경 끄기의 기술 (블랙커버 특별판) *배송주문 ..         마크 맨슨    1     1    2
11              82년생 김지영(오늘의 젊은 작가 13)-조남주 ..           조남주    2     1    2
12          